In [16]:
import pandas as pd
import datetime
import time
from nba_api.stats.endpoints import LeagueDashPlayerStats

In [17]:
# measure type: usage
# per mode: per game
# 2021-22 start date: October 19, 2021
# today: November 24, 2021
# count n games

# 2020-21 start date: December 22, 2020
# find second date for n games

In [18]:
TODAY = datetime.date.today()
CURR_SEASON_START = datetime.datetime.strptime('2021-10-19','%Y-%m-%d').date()
PAST_SEASON_START = datetime.datetime.strptime('2020-12-22','%Y-%m-%d').date()
PAST_SEASON_CHECKPOINT = PAST_SEASON_START + (TODAY - CURR_SEASON_START)

CURR_SEASON = '2021-22'
PAST_SEASON = '2020-21'

COLS = ['PLAYER_NAME', 'TEAM_ABBREVIATION', 'GP', 'MIN', 'PCT_FTA']

In [19]:
CURR_SEASON_STATS = LeagueDashPlayerStats(
    measure_type_detailed_defense='Usage',
    per_mode_detailed='PerGame',
    date_from_nullable=CURR_SEASON_START,
    date_to_nullable=TODAY
).get_data_frames()[0][COLS]

time.sleep(10)

In [20]:
PAST_SEASON_STATS = LeagueDashPlayerStats(
    measure_type_detailed_defense='Usage',
    per_mode_detailed='PerGame',
    date_from_nullable=PAST_SEASON_START,
    date_to_nullable=PAST_SEASON_CHECKPOINT,
    season=PAST_SEASON
).get_data_frames()[0][COLS]

time.sleep(10)

In [21]:
CURR_SEASON_FTA = LeagueDashPlayerStats(
    measure_type_detailed_defense='Base',
    per_mode_detailed='Per36',
    date_from_nullable=CURR_SEASON_START,
    date_to_nullable=TODAY,
    season=CURR_SEASON
).get_data_frames()[0][['PLAYER_NAME', 'TEAM_ABBREVIATION', 'GP', 'MIN', 'FTA']]

time.sleep(10)

In [22]:
PAST_SEASON_FTA = LeagueDashPlayerStats(
    measure_type_detailed_defense='Base',
    per_mode_detailed='Per36',
    date_from_nullable=PAST_SEASON_START,
    date_to_nullable=PAST_SEASON_CHECKPOINT,
    season=PAST_SEASON
).get_data_frames()[0][['PLAYER_NAME', 'TEAM_ABBREVIATION', 'GP', 'MIN', 'FTA']]


In [23]:
# min games 15
# min minutes
CURR_SEASON_STATS = CURR_SEASON_STATS[
    (CURR_SEASON_STATS['GP'] >= 10) &
    (CURR_SEASON_STATS['MIN'] >= 100)
].sort_values(by='PCT_FTA', ascending=False).iloc[:20]

PLAYERS_LIST = list(CURR_SEASON_STATS['PLAYER_NAME'])

In [24]:
PAST_SEASON_STATS = PAST_SEASON_STATS[
    PAST_SEASON_STATS['PLAYER_NAME'].isin(PLAYERS_LIST)
].sort_values(by='PCT_FTA', ascending=False)

PAST_SEASON_STATS

,PLAYER_NAME,TEAM_ABBREVIATION,GP,MIN,PCT_FTA
159,Giannis Antetokounmpo,MIL,17,561.0,0.600
439,Trae Young,ATL,18,614.0,0.504
24,Bam Adebayo,MIA,16,522.0,0.490
182,Ja Morant,MEM,5,146.0,0.484
101,DeMar DeRozan,SAS,17,577.0,0.476
266,Kevin Durant,BKN,15,551.0,0.453
294,Luka Doncic,DAL,18,623.0,0.452
390,Rudy Gobert,UTA,19,573.0,0.408
200,James Harden,BKN,16,609.0,0.401
123,Domantas Sabonis,IND,19,699.0,0.390


In [25]:
CURR_SEASON_STATS

,PLAYER_NAME,TEAM_ABBREVIATION,GP,MIN,PCT_FTA
157,Giannis Antetokounmpo,MIL,18,586.0,0.679
224,Jimmy Butler,MIA,16,546.0,0.504
98,DeMar DeRozan,CHI,20,695.0,0.495
348,Nikola Jokic,DEN,14,454.0,0.484
26,Bam Adebayo,MIA,16,524.0,0.461
389,Rudy Gobert,UTA,18,564.0,0.442
202,James Harden,BKN,19,665.0,0.436
183,Ja Morant,MEM,19,619.0,0.429
284,Kristaps Porzingis,DAL,12,365.0,0.426
18,Anthony Davis,LAL,19,665.0,0.411


In [26]:
# combine data and begin graphing
CURR_SEASON_STATS.rename(columns={'PCT_FTA': 'CURR_FTR'}, inplace=True)
CURR_SEASON_STATS.drop(columns=['TEAM_ABBREVIATION', 'GP', 'MIN'], inplace=True)

In [27]:
df = CURR_SEASON_STATS.merge(PAST_SEASON_STATS, left_on='PLAYER_NAME', right_on='PLAYER_NAME')
df.rename(columns={'PCT_FTA': 'PAST_FTR'}, inplace=True)
df['FTR_DIFF'] = df['PAST_FTR'] - df['CURR_FTR']
df.drop(columns=['TEAM_ABBREVIATION', 'GP', 'MIN'], inplace=True)
df.sort_values(by='FTR_DIFF', ascending=False, inplace=True)

In [28]:
# filter FTA dfs
CURR_SEASON_FTA = CURR_SEASON_FTA[CURR_SEASON_FTA['PLAYER_NAME'].isin(PLAYERS_LIST)]
PAST_SEASON_FTA = PAST_SEASON_FTA[PAST_SEASON_FTA['PLAYER_NAME'].isin(PLAYERS_LIST)]

CURR_SEASON_FTA.rename(columns={'FTA': 'CURR_FTA'}, inplace=True)
PAST_SEASON_FTA.rename(columns={'FTA': 'PAST_FTA'}, inplace=True)

df = df.merge(CURR_SEASON_FTA[['PLAYER_NAME', 'CURR_FTA']], left_on='PLAYER_NAME', right_on='PLAYER_NAME')
df = df.merge(PAST_SEASON_FTA[['PLAYER_NAME', 'PAST_FTA']], left_on='PLAYER_NAME', right_on='PLAYER_NAME')

df['FTA_DIFF'] = df['PAST_FTA'] - df['CURR_FTA']


In [29]:
df

,PLAYER_NAME,CURR_FTR,PAST_FTR,FTR_DIFF,CURR_FTA,PAST_FTA,FTA_DIFF
0,Trae Young,0.397,0.504,0.107,5.9,11.3,5.4
1,Luka Doncic,0.370,0.452,0.082,5.7,7.9,2.2
2,Ja Morant,0.429,0.484,0.055,6.5,7.6,1.1
3,Kevin Durant,0.403,0.453,0.050,6.8,7.9,1.1
4,Bam Adebayo,0.461,0.490,0.029,7.3,6.8,-0.5
5,Domantas Sabonis,0.377,0.390,0.013,4.5,5.7,1.2
6,DeMar DeRozan,0.495,0.476,-0.019,7.9,7.4,-0.5
7,Anthony Davis,0.411,0.377,-0.034,6.7,6.3,-0.4
8,Rudy Gobert,0.442,0.408,-0.034,7.5,6.3,-1.2
9,James Harden,0.436,0.401,-0.035,7.1,6.9,-0.2


In [30]:
df.to_csv('data/nba_ftr_diffs.csv')